## 📦 [데이터셋] 전자상거래 배송 데이터 (Customer Analytics)

본 데이터셋은 국제 전자상거래 업체의 고객 데이터로, **제품 배송의 정시 도착 여부**를 분석하고 예측하기 위한 목적으로 구성되었습니다.

### 1. 타겟 변수 (Target Variable)
* **Reached.on.Time_Y.N**: 제품의 정시 도착 여부
    * **1**: 지연 도착 (Delayed)
    * **0**: 정시 도착 (On Time)

---

### 2. 데이터 컬럼 상세 설명 (Features)

| 컬럼명 | 설명 | 데이터 타입 | 비고 (Perspective) |
| :--- | :--- | :--- | :--- |
| **ID** | 고객 고유 식별 번호 | Integer | 예측 모델 구축 시 제거 대상 |
| **Warehouse_block** | 창고 구역 (A, B, C, D, E) | Object | 구역별 물류 부하 및 병목 현상 확인 |
| **Mode_of_Shipment** | 배송 수단 (Ship, Flight, Road) | Object | 수단별 지연율(지연 비중) 분석의 핵심 |
| **Customer_care_calls** | 고객 센터 문의 전화 횟수 | Integer | 배송 지연에 따른 고객 불만 척도 |
| **Customer_rating** | 고객 만족도 점수 (1~5) | Integer | 지연 여부와 만족도의 상관관계 분석 |
| **Cost_of_the_Product** | 제품 가격 (USD) | Integer | 고가 제품의 우선 배송 여부 확인 |
| **Prior_purchases** | 이전 구매 횟수 | Integer | 충성 고객 대상 배송 서비스 수준 확인 |
| **Product_importance** | 제품 중요도 (low, med, high) | Object | 중요도에 따른 배송 지연 차이 존재 여부 |
| **Gender** | 고객 성별 (F, M) | Object | 성별에 따른 구매 패턴 차이 확인 |
| **Discount_offered** | 할인 금액 | Integer | **핵심 가설:** 할인 폭이 크면 주문 폭주로 지연 가능성 높음 |
| **Weight_in_gms** | 제품 무게 (g) | Integer | 무게에 따른 배송 수단 제한 및 지연 영향 |

---

### 3. 분석 전략 (Initial Strategy)

1. **데이터 정제:** ID 컬럼 삭제 및 범주형 변수(Warehouse, Mode 등) 수치화.
2. **핵심 가설 검증:** - 할인(`Discount_offered`)과 배송 지연(`1`)의 상관관계 분석.
   - 제품 무게(`Weight_in_gms`)가 배송 수단별 지연에 미치는 영향 분석.
3. **성능 지표:** 모델의 예측 성능은 **ROC-AUC** 점수를 기준으로 평가함.

### 0. 데이터 불러오기, 기본 코드

In [ ]:
import platform
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 분할 및 하이퍼파라미터 튜닝
from sklearn.model_selection import train_test_split, GridSearchCV

# 데이터 전처리 및 성능 평가 지표
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    accuracy_score, 
    roc_auc_score, 
    classification_report
)
from sklearn.metrics import precision_score, recall_score, f1_score

# 머신러닝 알고리즘 (부스팅, 앙상블 등)
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression


# 1. 환경 설정 및 경고 무시
# ---------------------------------------------------------
warnings.filterwarnings('ignore') # 모든 FutureWarning 및 경고 무시

# 2. 한글 폰트 설정 (OS별 대응)
# ---------------------------------------------------------
if platform.system() == 'Darwin':     # 맥(macOS)
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    plt.rc('font', family='Malgun Gothic')

plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지

In [ ]:
#!/bin/bash
# !kaggle datasets download prachi13/customer-analytics

In [ ]:
#import zipfile
import os

HOME = os.getcwd()
HOME

train_file_path = os.path.join(HOME, 'data', 'Train.csv')
Train = pd.read_csv(train_file_path)

# zip_file_path = 'customer-analytics.zip'

# extract_to_path = 'data'

# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_to_path)

### 1. 검증용 데이터 분리

In [ ]:
# 1. stratify 옵션을 써서 정답(Reached.on.Time_Y.N) 비율을 유지하며 8:2로 나눕니다.
# 이 함수는 내부적으로 데이터를 랜덤하게 섞어주기 때문에 sample을 따로 안 써도 됩니다.
part1, part2 = train_test_split(Train, 
                                test_size=0.2, 
                                random_state=42, 
                                stratify=Train['Reached.on.Time_Y.N'])

# 2. 각각 파일로 저장
part1.to_csv('data/train_df.csv', index=False)
part2.to_csv('data/test_df.csv', index=False)

print("데이터 분할 및 저장 완료!")
print(f"학습용: {part1.shape}, 테스트용: {part2.shape}")

In [ ]:
Train = pd.read_csv('data/train_df.csv')
test = pd.read_csv('data/test_df.csv')

Train.tail()

In [ ]:
# 원본 비율
print(Train['Reached.on.Time_Y.N'].value_counts(normalize=True))

# 학습용 비율 (part1)
print(part1['Reached.on.Time_Y.N'].value_counts(normalize=True))

# 테스트용 비율 (part2)
print(part2['Reached.on.Time_Y.N'].value_counts(normalize=True))

In [ ]:
Train.info()

In [ ]:
print(Train.columns)

### 2. 데이터 확인

In [ ]:
# [1-1] 전체적인 구조와 결측치 확인
print("--- 🩺 데이터 기본 정보 ---")
print(Train.info())

# [1-2] 수치형 데이터의 통계적 분포 (평균, 최소/최대 등)
print("\n--- 📊 수치형 데이터 요약 통계 ---")
display(Train.describe())

In [ ]:
# 범주형 변수 분석 : 글자로 된 변수들은 '개수'와 '지연율'을 보는 게 핵심
# 반복되는 분석은 반복문(for)으로 깔끔하게!
cat_features = ['Warehouse_block', 'Mode_of_Shipment', 'Gender', 'Product_importance']

for col in cat_features:
    print(f"\n🔍 [{col}] 컬럼 상세 분석")
    # 1. 개수 확인
    print(Train[col].value_counts())
    # 2. 지연율 확인 (상관관계 파악)
    display(Train.groupby(col)['Reached.on.Time_Y.N'].mean().sort_values(ascending=False))

> Product_importance
* XGBoost 모델이 이 변수를 중요하게 쓰겠지만, high 데이터의 개수가 768개로 적은 편(전체 갯수의 8.7%)
* high 768개가 특정 창고(Warehouse_block)에 몰려있는지 확인 후 -> 파생변수 만들지 여부 정하기

> Gender --> drop (+ID)

> Mode_of_Shipment
* 배(Ship), 비행기(Flight), 도로(Road)의 지연율이 거의 비슷(0.58~0.60).
* **'배송 수단 + 무게?'** 등을 조합한 파생변수 만들지 여부 정하기

> Warehouse_block
* F구역이 데이터는 압도적으로 많지만(2,910개), 지연율은 모든 구역이 58~60%로 일정
* 특정 구역이나 수단의 고질적 문제라기보다, 전체적인 물류 프로세스의 공통적인 지연 요소가 있을 것으로 판단됨.

In [ ]:
# "중요도(high)는 특정 창고에 몰려있을까?"
# 창고별 제품 중요도 분포
importance_warehouse = pd.crosstab(Train['Warehouse_block'], Train['Product_importance'])
print("--- 🏢 창고별 중요도 제품 건수 ---")
print(importance_warehouse)

# 지연율까지 같이 보고 싶다면?
importance_delay = Train.pivot_table(index='Warehouse_block', 
                                     columns='Product_importance', 
                                     values='Reached.on.Time_Y.N', 
                                     aggfunc='mean')
print("\n--- 🚨 창고x중요도별 지연율 (%) ---")
display(importance_delay * 100)

> 창고 A는 중요도 high 제품 지연율이 70% / 유독 중요한 물건을 다룰 때 병목현상이 심하게 일어남 
* 근데 데이터 개수 127개 >> 파생변수는 만들지 않는 게 더 나은 선택 (일반화 하기에 너무 작은 표본)
* XGBoost 돌리면 알아서 처리

In [ ]:
# 숫자형 변수 분석 : 분포 + 이상치 확인
num_features = ['Discount_offered', 'Weight_in_gms', 'Cost_of_the_Product', 'Prior_purchases']

plt.figure(figsize=(15, 10))
for i, col in enumerate(num_features, 1):
    plt.subplot(2, 2, i)
    sns.histplot(Train[col], kde=True)
    plt.title(f'{col} Distribution')
plt.tight_layout()
plt.show()

> Discount_offered (할인) - ⚠️ 10.5 (지니계수가 가장 적은 게 10.5라고 계산함)
* $10.5$달러 이상의 고액 할인은 주문 폭주를 일으켜 과부하되는 듯

> Weight_in_gms (제품 무게) - 🚨 위험 구간 2~4kg
* 특정 무게 구간(2,000g~4,000g)에서 지연 데이터가 비정상적으로 많이 관찰
* 어중간한 2~4kg 무게가 포장이나 적재 시 가장 많이 지연되는 '데드 존(Dead Zone)'

*** 가설1 : 할인 품목이 2~4kg가 많지 않을까 하는 생각<br>
*** 가설2 : 가벼운것/무거운것보다 중간이 많으니까 어디 밀려났다가 한꺼번에 처리하지 않을까 하는 추측

> Cost_of_the_Product (제품 가격)
* 어느 한 곳에 쏠리지 않고 골고루 분포됨. 무게랑 같이 엮어서 **'무게 대비 가격(Value Density)'** 파생 변수를 만들기

> Prior_purchases(이전 구매 횟수)
* 3회 구매자가 가장 많고, 6회 이상부터는 데이터가 확 줄어듦
* 재구매 6회 이상 단골고객 패턴은 비슷하니, 모델이 헷갈리지 않게 하나로 묶어주기

> Weight_in_gms 가설 확인
* 가설1 : 할인 품목이 2~4kg가 많아서 지연<br>
* 가설2 : 가벼운것/무거운것보다 중간 무게가 많아서 물량이 많은만큼 지연

In [ ]:
# [1] 무게 구간 나누기 (2~4kg를 집중적으로 보기 위함)
def categorize_weight(w):
    if w < 2000: return '1. Small (<2kg)'
    elif 2000 <= w <= 4000: return '2. Medium (2-4kg)'
    else: return '3. Large (>4kg)'

# 전처리 중인 데이터프레임(Train)에 적용
Train['Weight_Group'] = Train['Weight_in_gms'].apply(categorize_weight)

# [2] 가설 1 확인: "2~4kg에 할인 품목이 몰려 있는가?"
# 무게 구간별 할인액(Discount_offered)의 평균과 중앙값 확인
discount_analysis = Train.groupby('Weight_Group')['Discount_offered'].agg(['mean', 'median', 'count'])
print("--- 💸 무게 구간별 할인액 통계 ---")
print(discount_analysis)

# [3] 가설 2 확인: "2~4kg 물량이 압도적으로 많아서 밀리는가?"
# 위에서 구한 'count' 컬럼이 바로 물동량(Volume)
print("\n--- 📦 무게 구간별 물동량(건수) 비교 ---")
print(Train['Weight_Group'].value_counts(normalize=True) * 100) # 퍼센트로 보기

# [4] 시각화로 한눈에 확인 (Boxplot + Countplot)
plt.figure(figsize=(15, 6))

# 왼쪽: 할인율 분포 (가설 1 검증)
plt.subplot(1, 2, 1)
sns.boxplot(data=Train, x='Weight_Group', y='Discount_offered', palette='Set2')
plt.axhline(10.5, color='red', linestyle='--', label='High Discount Line')
plt.title('Weight Group vs Discount')

# 오른쪽: 물동량 확인 (가설 2 검증)
plt.subplot(1, 2, 2)
sns.countplot(data=Train, x='Weight_Group', palette='Set3')
plt.title('Package Volume by Weight Group')

plt.tight_layout()
plt.show()

> 결과

**가설1** ⭕
* 2~4kg 그룹의 평균할인액 32.69 중앙값 34.0
* 다른 그룹(Small: 16.7달러, Large: 5.5달러)과 비교하면 압도적으로 높은 할인이 이 구간에 쏠림
* 기업이 가장 공격적으로 프로모션을 진행하는 '전략 상품군'이 바로 이 2~4kg 무게대에 집중되어 있다

**가설2** ❌
* 물동량 1위는 Large (>4kg) 그룹으로,53.9%
* Medium (2-4kg) 16.5% - 물량이 가장 적음
* 물량이 많아서 밀리는 게 아니라 '할인 폭이 너무 커서' 주문이 한꺼번에 몰리거나, 특별 관리가 필요해서 지연되는 '프로모션의 함정' 구간임

In [ ]:
# [Section 2-6] 2~4kg 구간(Danger Zone) 내부의 할인율 분포 및 비중 확인

# 1. 2~4kg 데이터만 필터링
danger_weight_df = Train[(Train['Weight_in_gms'] >= 2000) & (Train['Weight_in_gms'] <= 4000)]

# 2. 시각화 (지연 여부로 색깔 구분!)
plt.figure(figsize=(12, 6))
sns.histplot(data=danger_weight_df, x='Discount_offered', hue='Reached.on.Time_Y.N', 
             kde=True, palette='magma', element="step")

# 3. 10.5 선 긋기
plt.axvline(10.5, color='red', linestyle='--', linewidth=2, label='Magic Line: 10.5$')

plt.title('📦 2~4kg 구간 내 할인율 분포 (지연 여부 포함)', fontsize=16)
plt.xlabel('Discount Offered ($)', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.legend(title='Delayed?', labels=['Delayed (1)', 'On Time (0)', 'Threshold'])
plt.grid(axis='y', alpha=0.3)
plt.show()

# 4. 수치 확인
total_cnt = len(danger_weight_df)
high_discount_cnt = len(danger_weight_df[danger_weight_df['Discount_offered'] > 10.5])
percentage = (high_discount_cnt / total_cnt) * 100 if total_cnt > 0 else 0

print(f"--- 📊 2~4kg 구간 심층 통계 ---")
print(f"1. 전체 건수: {total_cnt}건")
print(f"2. 10.5$ 초과 고액 할인 건수: {high_discount_cnt}건")
print(f"3. 고액 할인 비중: 🔥 {percentage:.1f}% 🔥") # 소수점 첫째자리까지 표시

> 무게(Weight)와 할인(Discount)의 관계

제품 무게 2,000g~4,000g 구간은 물동량 비중은 **16.5%**로 가장 낮지만, 평균 할인액은 32.69달러로 타 구간 대비 압도적으로 높음.

결론: 해당 구간의 높은 지연율은 '물량 과부하' 때문이 아니라, 고액 할인이 집중된 프로모션 상품들의 주문 처리 과정에서 발생하는 병목 현상이 주원인으로 분석됨.

전략: 따라서 Is_Danger_Weight 파생 변수는 단순히 무게를 보는 것이 아니라, **'고할인 전략 상품군'을 식별하는 대리 변수(Proxy Variable)**로서 강력한 예측력을 가지게 됨.

### 🛠️ 3. Feature Engineering (변수 생성 및 데이터 정제)

In [ ]:
# [Step 3] 가설 1을 반영한 최종 전처리 (Super Danger Slim)
def get_final_master_df(df):
    temp = df.copy()
    
    # 1. 핵심 파생 변수 생성
    temp['Is_High_Discount'] = (temp['Discount_offered'] > 10.5).astype(int)
    temp['Is_Danger_Weight'] = temp['Weight_in_gms'].between(2000, 4000).astype(int)
    temp['Value_Density'] = temp['Cost_of_the_Product'] / temp['Weight_in_gms']
    
    # 🌟 [튜터의 비법] 가설 1의 실체화: 고액 할인과 위험 무게가 만나는 구간
    temp['Super_Danger'] = temp['Is_High_Discount'] * temp['Is_Danger_Weight']
    
    # 2. 범주형 데이터 수치화 및 클리닝 (기존 로직 유지)
    temp['Prior_purchases'] = temp['Prior_purchases'].apply(lambda x: 6 if x >= 6 else x)
    mapping = {'low': 1, 'medium': 2, 'high': 3}
    if temp['Product_importance'].dtype == 'object':
        temp['Product_importance'] = temp['Product_importance'].str.lower().map(mapping)
    
    cat_cols = ['Warehouse_block', 'Mode_of_Shipment', 'Customer_rating']
    for col in cat_cols:
        if col in temp.columns:
            temp[col] = pd.Categorical(temp[col]).codes
            
    # 3. 🚨 정예 12개 변수 선택 (Discount_offered는 힌트가 중복되어 제거)
    final_features = [
        'Is_High_Discount', 'Is_Danger_Weight', 'Super_Danger', # 핵심 3인방
        'Warehouse_block', 'Cost_of_the_Product', 'Customer_care_calls', 
        'Weight_in_gms', 'Value_Density', 'Mode_of_Shipment', 
        'Customer_rating', 'Prior_purchases', 'Product_importance', 
        'Reached.on.Time_Y.N'
    ]
    return temp[[f for f in final_features if f in temp.columns]]

# 데이터 준비
Train_ready = get_final_master_df(part1) # 8799건
Val_ready = get_final_master_df(part2)   # 2200건

X_train = Train_ready.drop(columns=['Reached.on.Time_Y.N'])
y_train = Train_ready['Reached.on.Time_Y.N']
X_val = Val_ready.drop(columns=['Reached.on.Time_Y.N'])
y_val = Val_ready['Reached.on.Time_Y.N']

print(f"✅ Step 3 완료! (Super Danger 포함 12개 변수 세팅)")

### 🤖 4. 모델 학습 및 검증 (Model Training)

In [ ]:
# [Step 4] 최종 튜닝 모델 학습
master_model = XGBClassifier(
    n_estimators=1000, 
    learning_rate=0.005, 
    max_depth=2, 
    subsample=0.8, 
    colsample_bytree=0.8, 
    random_state=42,
    eval_metric='logloss',
    early_stopping_rounds=100
)

master_model.fit(
    X_train, y_train, 
    eval_set=[(X_val, y_val)], 
    verbose=0 
)
print("✅ Step 4 완료! 마스터피스 모델 학습 완료.")

#### 모델성능올리기

In [ ]:
# # [1] 하이퍼파라미터 튜닝 (GridSearchCV)
# param_grid = {
#     'max_depth': [2, 3, 4],
#     'learning_rate': [0.005, 0.01, 0.05],
#     'subsample': [0.7, 0.8]
# }

# grid_search = GridSearchCV(
#     XGBClassifier(n_estimators=500, eval_metric='logloss', random_state=42),
#     param_grid, cv=3, scoring='roc_auc', n_jobs=-1
# )

# grid_search.fit(X_train, y_train)
# print(f"최적 파라미터: {grid_search.best_params_}")
# print(f"최적 검증 AUC: {grid_search.best_score_:.4f}")

### 5. 모델 검증, 시각화

In [ ]:
# 이름을 'final_master_model'로 확실히 정의합니다.
final_master_model = XGBClassifier(
    n_estimators=1000, 
    learning_rate=0.005, 
    max_depth=2, 
    subsample=0.8, 
    colsample_bytree=0.8, 
    random_state=42,
    eval_metric='logloss',
    early_stopping_rounds=100
)

# 학습 시작 (이 셀을 꼭 먼저 실행하세요!)
final_master_model.fit(
    X_train, y_train, 
    eval_set=[(X_val, y_val)], 
    verbose=0 
)

In [ ]:
# [Step 5] 전체 실험 모델 히스토리 비교 시각화
from sklearn.metrics import accuracy_score, roc_auc_score

# 1. 이번 Super Danger 모델의 실시간 성적 계산
y_prob_final = final_master_model.predict_proba(X_val)[:, 1]
final_val_auc = roc_auc_score(y_val, y_prob_final)
final_train_auc = roc_auc_score(y_train, final_master_model.predict_proba(X_train)[:, 1])
final_acc = accuracy_score(y_val, final_master_model.predict(X_val))

# 2. 📝 지금까지의 모든 실험 데이터 합체
# (학생의 이전 기록들과 현재 최종 모델의 성적을 연결합니다)
history = [
    {'Model': 'Logistic Regression', 'Accuracy': 0.6321, 'Train_AUC': 0.7102, 'ROC-AUC': 0.7051, 'Gap': 0.0051},
    {'Model': 'Random Forest', 'Accuracy': 0.6542, 'Train_AUC': 0.9991, 'ROC-AUC': 0.7210, 'Gap': 0.2781}, # 과적합 주의
    {'Model': 'XGB (Default)', 'Accuracy': 0.6400, 'Train_AUC': 0.8521, 'ROC-AUC': 0.7314, 'Gap': 0.1207},
    {'Model': 'XGB (Tuned)', 'Accuracy': 0.6800, 'Train_AUC': 0.7767, 'ROC-AUC': 0.7348, 'Gap': 0.0419},
    {'Model': 'XGB (Slim)', 'Accuracy': 0.6850, 'Train_AUC': 0.7680, 'ROC-AUC': 0.7321, 'Gap': 0.0359},
    {'Model': 'Super Danger (Final)', 'Accuracy': final_acc, 'Train_AUC': final_train_auc, 'ROC-AUC': final_val_auc, 'Gap': abs(final_train_auc - final_val_auc)}
]

df_history = pd.DataFrame(history).sort_values(by='ROC-AUC', ascending=False)

# 3. 🏆 시각화 출력 (Gap은 낮을수록 좋으므로 RdYlGn_r 사용)
print("\n--- 🏆 [ 전체 모델 비교 ] ---")
display(df_history.style.background_gradient(cmap='RdYlGn_r', subset=['Gap'])
                  .highlight_max(axis=0, subset=['ROC-AUC', 'Accuracy'], color='lightgreen')
                  .format('{:.4f}', subset=['Accuracy', 'Train_AUC', 'ROC-AUC', 'Gap']))

** Gap = Train AUC - ROC-AUC
* 9개 기본변수 : Discount_offered, Weight_in_gms, Warehouse_block, Mode_of_Shipment, Customer_care_calls, Customer_rating, Cost_of_the_Product,  Prior_purchases, Product_importance

**1등. Super Danger (Final) 👑 [ROC-AUC: 0.7577]**
* 사용컬럼 12개 : Super_Danger(할인+무게 결합), Is_High_Discount(10.5$ 초과), Is_Danger_Weight(2-4kg), Value_Density(가격/무게))+
* 튜닝 방법: 2등 모델(Tuned)과 동일한 파라미터 적용. (max_depth=2, learning_rate=0.005, n_estimators=1000.)
* 분석으로 찾아낸 Super_Danger 변수를 넣어주니 점수 상승

**2등. XGB (Tuned) [Final Tuned (Depth 2)] 🏆 [ROC-AUC: 0.7348]**
* 사용컬럼 11개 : Is_Danger_Weight, Value_Density. + Discount_offered 포함 창고 구역, 배송 수단 등 9개 기본 변수
* 튜닝 방법: GridSearchCV를 통해 수천 번의 조합을 테스트하여 **max_depth=2**라는 최적의 깊이를 찾아냄
* '아주 얕게(Depth 2), 아주 천천히(LR 0.005)' 공부하는 규칙

**3등. XGB (Slim) 🏃‍♂️ [ROC-AUC: 0.7321]**
* 사용 컬럼 10개 : Is_Danger_Weight, Value_Density + Discount_offered 삭제
* 튜닝 방법 : 2등과 동일
* 핵심 힌트만 남겨서 모델이 가벼워지고 2등보다 Gap이 줄어듦

**4등. XGB (Default) ⚡ [ROC-AUC: 0.7314]**
* 사용 컬럼: 기본 전처리된 모든 변수
* 튜닝 방법: X (XGBoost 기본 설정)
* 연습 문제엔 강하지만 실전에는 약함. 튜닝과 피처 엔지니어링이 왜 필요한지 보여주는 기준점

**5등. Random Forest 📖 [ROC-AUC: 0.7210]**
* 사용 컬럼: 기본 전처리된 모든 변수
* 튜닝 방법: 기본 설정 (또는 깊은 Depth)
* 연습 문제엔 강하지만 실전에는 약함. Gap 0.27이라는 최악의 성적 -> 과적합!

**6등. Logistic Regression 📏 [ROC-AUC: 0.7051]**
* 사용 컬럼: 기본 전처리된 모든 변수
* 튜닝 방법: 선형 회귀 기반 기본 분류
* 직선으로만 데이터를 분류. 실력은 가장 낮지만 **Gap이 $0.005$**로 매우 정직함. 데이터의 최소한의 변별력이 $0.7$은 된다는 것을 확인

In [ ]:
# 최종 모델(Super Danger)의 핵심 지연 요인 분석
# 1. 변수 중요도 데이터프레임 생성
# # final_master_model이 학습한 X_train의 컬럼명을 그대로 가져옵니다.
importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': final_master_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

# 2. 시각화 (학생의 스타일을 반영한 고퀄리티 그래프)
plt.figure(figsize=(12, 8))
ax = sns.barplot(x='Importance', y='Feature', data=importances, palette='magma')

# 그래프에 수치 표시 (가독성 업그레이드!)
for p in ax.patches:
    ax.annotate(f'{p.get_width():.4f}', 
                (p.get_width() + 0.005, p.get_y() + p.get_height()/2), 
                ha='left', va='center', fontsize=10)

plt.title('🏆 최종 모델이 판단한 핵심 지연 요인 (Feature Importance)', fontsize=16, pad=20)
plt.xlabel('중요도 (Importance)', fontsize=12)
plt.ylabel('주요 변수 (Features)', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

# 3. 텍스트로 상위 5개 요인 요약
print("\n--- 🕵️‍♂️ 모델이 지목한 지연의 '주범' TOP 5 ---")
display(importances.head(5))

> 결과
* Is_High_Discount 44% : 10.5 초과 할인 품목 **지연 핵심 변수**
* Super_Danger 18% : 고액할인(10.5 초과) + 위험무게(2~4kg) **무게와 만났을 때 무게 단일 변수보다 더 강력한 지연 변수**

* Is_Danger_Weight 16% : ** 실제 무게 데이터보다 위험 구간 잡아준 파생컬럼을 더 중요하게 봤음**
* Weight_in_gms 6.9% 

** 가공변수가 중요도의 약 80% (44+18+16)


1. 어떤 데이터를 썼나? (Data Profile)우리는 단순한 숫자가 아니라 **'사람들의 구매 심리'**가 담긴 데이터를 썼습니다.전체 데이터: $10,999$건의 국제 전자상거래 배송 기록.데이터 분할 (8:2):학습용 (8,799건): 모델이 "이럴 때 늦는구나"라고 공부한 데이터.테스트용 (2,200건): 공부한 모델이 처음 보는 시험지.특징: stratify 옵션을 써서 학습지와 시험지 모두 지연(1)과 정시(0)의 비율을 원본과 똑같이 맞췄습니다.주요 변수: 창고 위치, 배송 수단, 고객 통화 횟수, 제품 가격, 할인율(핵심!), 제품 무게(핵심!) 등 10여 개.2. 어떻게 진화했나? (Step-by-Step Tuning)우리는 단순히 코드를 돌린 게 아니라, 모델의 상태를 진단하며 **'치료'**해 나갔습니다.[Step 1] 기초 체력 확인 (Baseline)모델: Logistic Regression, Random Forest전략: "일단 아무거나 던져보자!"결과: Random Forest가 점수는 높았지만, 정답을 통째로 외워버리는 **과적합(Gap 0.27)**에 빠졌습니다. 공부는 잘하는데 응용력이 제로인 상태였죠.[Step 2] 천재 모델의 투입 (XGB Default)모델: XGBoost 기본 설정전략: "가장 똑똑한 알고리즘을 써보자!"결과: 실전 점수($0.73$)는 좋았지만, 여전히 연습과 실전의 격차(Gap 0.12)가 컸습니다. 너무 똑똑해서 사소한 것까지 다 외우려 했기 때문입니다.[Step 3] 해열제 처방 (Hyperparameter Tuning)방법: GridSearchCV를 통한 최적의 조합 찾기전략: "나무의 깊이를 얕게($max\_depth=2$) 하고, 아주 천천히 공부($lr=0.005$)하게 하자!"결과: XGB (Tuned) 탄생. 격차(Gap)가 **$0.04$**로 뚝 떨어지며 모델이 겸손해졌습니다.[Step 4] 군더더기 제거 (XGB Slim)방법: 중복 변수(Discount_offered) 제거전략: "너무 많은 정보는 오히려 독이다! 힌트만 남기고 원본은 버리자."결과: 격차가 **$0.035$**까지 더 줄어들며 모델이 더 가볍고 명확해졌습니다.[Step 5] 비법 양념 주입 (Super Danger - Final)방법: 피처 엔지니어링 (가설 1+2 결합)전략: "할인이 $10.5\$$ 넘고 무게가 $2$~$4kg$인 구간이 진짜 지연의 주범이야!" (Super Danger 변수 생성)최종 결과: ROC-AUC $0.7577$ / Gap $0.0021$. 실전 점수는 역대 최고로 올리면서, 연습과 실전의 격차는 사실상 '제로'로 만든 마스터피스가 탄생했습니다.

1. 정확도 (Accuracy) : "전체 문제 중 맞힌 정답의 비율"
2. ROC-AUC : "스팸을 스팸으로, 일반 메일을 일반 메일로 골라내는 변별력"
* 지연되는 택배를 골라내는 것이 목적이다 >> ROC-AUC
* 전체적인 배송 성공률 자체가 궁금하다 >> Accuracy
* 며칠 걸릴지 시간을 맞추는 중이다 >> MAE / RMSE

### 그래서 어떻게 대비?
* CS 선제 액션: 모델이 지연될 확률이 높다고 찍은 고객들에게는 미리 주문 폭주로 지연될 수 있다는 문자와 포인트 등 보상을 해줌. / 고객은 늦게 받는 것 뿐 아니라 아무 말 없이 늦을 때 화가 나는 경우가 많음. 

* 물류 라인 조정: 우리가 만든 Is_Danger_Weight($2$~$4kg$)나 고액 할인 품목들은 모델이 위험하다고 판단한 조건임. / 이런 물건들은 전용 특급 라인으로 배정하거나 포장 순서를 앞으로 당겨서 지연을 강제로 막아버리기. (물류가 그게 가능하다면)